# Actividad - Proyecto práctico


> La actividad se desarrollará en grupos pre-definidos de 2-3 alumnos. Se debe indicar los nombres en orden alfabético (de apellidos). Recordad que esta actividad se corresponde con un 30% de la nota final de la asignatura. Se debe entregar entregar el trabajo en la presente notebook.
*   Alumno 1: Gil Garcera, Javier 
*   Alumno 2: Palomares Mateo, Abel 
*   Alumno 3: Serrano López, Francisco Rubén 
*   Alumno 4: Vegas Romero, David 







---
## **PARTE 1** - Instalación y requisitos previos

> Las prácticas han sido preparadas para poder realizarse en el entorno de trabajo de Google Colab. Sin embargo, esta plataforma presenta ciertas incompatibilidades a la hora de visualizar la renderización en gym. Por ello, para obtener estas visualizaciones, se deberá trasladar el entorno de trabajo a local. Por ello, el presente dosier presenta instrucciones para poder trabajar en ambos entornos. Siga los siguientes pasos para un correcto funcionamiento:
1.   **LOCAL:** Preparar el enviroment, siguiendo las intrucciones detalladas en la sección *1.1.Preparar enviroment*.
2.  **AMBOS:** Modificar las variables "mount" y "drive_mount" a la carpeta de trabajo en drive en el caso de estar en Colab, y ejecturar la celda *1.2.Localizar entorno de trabajo*.
3. **COLAB:** se deberá ejecutar las celdas correspondientes al montaje de la carpeta de trabajo en Drive. Esta corresponde a la sección *1.3.Montar carpeta de datos local*.
4.  **AMBOS:** Instalar las librerías necesarias, siguiendo la sección *1.4.Instalar librerías necesarias*.


---
### 1.1. Preparar enviroment (solo local)



> Para preparar el entorno de trabajo en local, se han seguido los siguientes pasos:
1. En Windows, puede ser necesario instalar las C++ Build Tools. Para ello, siga los siguientes pasos: https://towardsdatascience.com/how-to-install-openai-gym-in-a-windows-environment-338969e24d30.
2. Instalar Anaconda
3. Siguiendo el código que se presenta comentado en la próxima celda: Crear un enviroment, cambiar la ruta de trabajo, e instalar librerías básicas.


```
conda create --name miar_rl python=3.8
conda activate miar_rl
cd "PATH_TO_FOLDER"
conda install git
pip install jupyter
```


4. Abrir la notebook con *jupyter-notebook*.



```
jupyter-notebook
```


---
### 1.2. Localizar entorno de trabajo: Google colab o local

In [2]:
# ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
mount='/content/gdrive'
drive_root = mount + "/My Drive/08_MIAR/actividades/proyecto practico"

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

---
### 1.3. Montar carpeta de datos local (solo Colab)

In [3]:
# Switch to the directory on the Google Drive that you want to use
import os
if IN_COLAB:
  print("We're running Colab")

  if IN_COLAB:
    # Mount the Google Drive at mount
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root
# Verify we're in the correct working directory
%pwd
print("Archivos en el directorio: ")
print(os.listdir())

Archivos en el directorio: 
['dqn_SpaceInvaders-v0_weights_250000.h5f.index', 'dqn_SpaceInvaders-v0_weights_1000000.h5f.data-00000-of-00001', '.jupyter', 'dqn_SpaceInvaders-v0_weights_1750000.h5f.data-00000-of-00001', 'dqn_SpaceInvaders-v0_log.json', 'alternativa_google_collab.txt', 'dqn_SpaceInvaders-v0_weights_500000.h5f.data-00000-of-00001', 'Proyecto_práctico.ipynb', 'dqn_SpaceInvaders-v0_weights_750000.h5f.data-00000-of-00001', 'dqn_SpaceInvaders-v0_weights_1750000.h5f.index', 'dqn_SpaceInvaders-v0_weights.h5f.index', 'dqn_SpaceInvaders-v0_weights_1500000.h5f.data-00000-of-00001', 'dqn_SpaceInvaders-v0_weights_1250000.h5f.data-00000-of-00001', 'dqn_SpaceInvaders-v0_weights_500000.h5f.index', 'dqn_SpaceInvaders-v0_weights_1000000.h5f.index', 'Proyecto_práctico_local.ipynb', 'dqn_SpaceInvaders-v0_weights_1500000.h5f.index', 'README.md', '.ipynb_checkpoints', '.git', 'dqn_SpaceInvaders-v0_weights.h5f.data-00000-of-00001', 'dqn_SpaceInvaders-v0_weights_1250000.h5f.index', '.gitignore'

---
### 1.4. Instalar librerías necesarias

In [11]:
if IN_COLAB:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install keras-rl2==1.0.5
  %pip install tensorflow==2.8
  %pip install tensorflow==2.12.1
else:
  %pip install numpy==1.23.5
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install pyglet==1.5.0
  %pip install h5py==3.10.0
  %pip install Pillow==9.5.0
  %pip install keras-rl2==1.0.5
  %pip install Keras==2.12.0
  %pip install tensorflow==2.12.1
  %pip install torch==2.0.1
  %pip install agents==1.4.0

---
## **PARTE 2**. Enunciado

Consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será _SpaceInvaders-v0_ y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, el requisito mínimo será alcanzado cuando el agente consiga una **media de recompensa por encima de 20 puntos en modo test**. Por ello, esta media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este proyecto práctico consta de tres partes:

1.   Implementar la red neuronal que se usará en la solución
2.   Implementar las distintas piezas de la solución DQN
3.   Justificar la respuesta en relación a los resultados obtenidos

**Rúbrica**: Se valorará la originalidad en la solución aportada, así como la capacidad de discutir los resultados de forma detallada. El requisito mínimo servirá para aprobar la actividad, bajo premisa de que la discusión del resultado sera apropiada.

IMPORTANTE:

* Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.
* Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).
* Se deberá entregar unicamente el notebook y los pesos del mejor modelo en un fichero .zip, de forma organizada.
* Cada alumno deberá de subir la solución de forma individual.

---
## **PARTE 3**. Desarrollo y preguntas

#### Importar librerías

In [4]:
from __future__ import division

from PIL import Image
import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.optimizers.legacy import Adam

import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint
import tensorflow as tf


import torch
print("PyTorch:", torch.__version__)
print("¿GPU disponible?:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Nombre de GPU:", torch.cuda.get_device_name(0))

print("TensorFlow:", tf.__version__)
print("GPUs detectadas:", tf.config.list_physical_devices('GPU'))
torch.cuda.is_available()
tf.config.list_physical_devices('GPU')
tf.debugging.set_log_device_placement(True)

2025-06-28 22:58:32.414287: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-06-28 22:58:32.414304: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


PyTorch: 2.0.1+cu117
¿GPU disponible?: True
Nombre de GPU: NVIDIA GeForce RTX 4060 Laptop GPU
TensorFlow: 2.5.3
GPUs detectadas: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2025-06-28 22:58:33.745126: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2025-06-28 22:58:33.745248: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-06-28 22:58:33.745422: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 4060 Laptop GPU computeCapability: 8.9
coreClock: 1.89GHz coreCount: 24 deviceMemorySize: 7.75GiB deviceMemoryBandwidth: 238.45GiB/s
2025-06-28 22:58:33.745456: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2025-06-28 22:58:33.745478: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2025-06-28 22:58:33.745491: I tensorflow/stream_ex

#### Configuración base

In [5]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [6]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

1. Implementación de la red neuronal

In [7]:
print("Numero de acciones disponibles:" + str(nb_actions))
print("Formato de las observaciones:")
#env.observation_space
## Next, we build a very simple model.
#model = Sequential()
#model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
#model.add(Dense(16))
#model.add(Activation('relu'))
#model.add(Dense(16))
#model.add(Activation('relu'))
#model.add(Dense(16))
#model.add(Activation('relu'))
#model.add(Dense(nb_actions))
#model.add(Activation('linear'))
#print(model.summary())


# Next, we build our model. We use the same model that was described by Mnih et al. (2015).
input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE
model = Sequential()
print(K.image_data_format())
if K.image_data_format() == 'channels_last':
    # (width, height, channels)
    model.add(Permute((2, 3, 1), input_shape=input_shape))
elif K.image_data_format() == 'channels_first':
    # (channels, width, height)
    model.add(Permute((1, 2, 3), input_shape=input_shape))
else:
    raise RuntimeError('Unknown image_dim_ordering.')

model.add(Convolution2D(32, (8, 8), strides=(4, 4)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3), strides=(1, 1)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())


Numero de acciones disponibles:6
Formato de las observaciones:
channels_last
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute (Permute)            (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 20, 20, 32)        8224      
_________________________________________________________________
activation (Activation)      (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
activation_1 (Activation)    (None, 9, 9, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          36928     
_____________________________________________

In [13]:
"""# Let's define the memory for storing the experience
memory = SequentialMemory(limit=50000, window_length=1)

# Define the policy that our agent will follow
policy = BoltzmannQPolicy()

# Define the agent
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory,
               nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)

adam_obj = Adam(learning_rate=1e-3)
dqn.compile(adam_obj, metrics=['mae'])

# Train the agent
dqn.fit(env, nb_steps=50000, visualize=False, verbose=2)

# After training is done, we save the final weights.
dqn.save_weights('dqn_{}_weights.h5f'.format(ENV_NAME), overwrite=True)
# Finally, evaluate our algorithm for 5 episodes.
dqn.load_weights('dqn_{}_weights.h5f'.format(ENV_NAME))
dqn.test(env, nb_episodes=10, visualize=False)
"""

"# Let's define the memory for storing the experience\nmemory = SequentialMemory(limit=50000, window_length=1)\n\n# Define the policy that our agent will follow\npolicy = BoltzmannQPolicy()\n\n# Define the agent\ndqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory,\n               nb_steps_warmup=10,\n               target_model_update=1e-2, policy=policy)\n\nadam_obj = Adam(learning_rate=1e-3)\ndqn.compile(adam_obj, metrics=['mae'])\n\n# Train the agent\ndqn.fit(env, nb_steps=50000, visualize=False, verbose=2)\n\n# After training is done, we save the final weights.\ndqn.save_weights('dqn_{}_weights.h5f'.format(ENV_NAME), overwrite=True)\n# Finally, evaluate our algorithm for 5 episodes.\ndqn.load_weights('dqn_{}_weights.h5f'.format(ENV_NAME))\ndqn.test(env, nb_episodes=10, visualize=False)\n"

In [ ]:
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)
processor = AtariProcessor()

policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps',
                              value_max=1., value_min=.1, value_test=.05,
                              nb_steps=1000000)
dqn = DQNAgent(model=model, nb_actions=nb_actions, policy=policy,
               memory=memory, processor=processor,
               nb_steps_warmup=50000, gamma=.99,
               target_model_update=10000,
               train_interval=4)
dqn.compile(Adam(learning_rate=.00025), metrics=['mae'])

# Training part
weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
checkpoint_weights_filename = 'dqn_' + env_name + '_weights_{step}.h5f'
log_filename = 'dqn_{}_log.json'.format(env_name)
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=250000)]
callbacks += [FileLogger(log_filename, interval=100)]

2025-06-28 22:58:45.009522: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-28 22:58:45.010506: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-06-28 22:58:45.010693: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 4060 Laptop GPU computeCapability: 8.9
coreClock: 1.89GHz coreCount: 24 deviceMemorySize: 7.75GiB deviceMemoryBandwidth: 238.45GiB/s
2025-06-28 22:58:45.010741: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had ne

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9



2. Implementación de la solución DQN

In [15]:
dqn.fit(env, callbacks=callbacks, nb_steps=1750000, log_interval=10000, visualize=False)

dqn.save_weights(weights_filename, overwrite=True)

Training for 1750000 steps ...
Interval 1 (0 steps performed)
   43/10000 [..............................] - ETA: 24s - reward: 0.0000e+00

/home/fraserlp/anaconda3/envs/miar_rl/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


10000/10000 [==============================] - 20s 2ms/step - reward: 0.0131
14 episodes - episode_reward: 8.786 [6.000, 15.000] - ale.lives: 2.014

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 19s 2ms/step - reward: 0.0140
16 episodes - episode_reward: 9.250 [6.000, 14.000] - ale.lives: 2.182

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 19s 2ms/step - reward: 0.0145
13 episodes - episode_reward: 10.692 [5.000, 24.000] - ale.lives: 2.153

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 19s 2ms/step - reward: 0.0138
15 episodes - episode_reward: 9.067 [4.000, 13.000] - ale.lives: 2.225

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 20s 2ms/step - reward: 0.0118
14 episodes - episode_reward: 8.571 [4.000, 15.000] - ale.lives: 2.188

Interval 6 (50000 steps performed)
10000/10000 [==============================] - 145s 14ms/step - reward: 0.0121


In [16]:
# Testing part to calculate the mean reward
weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
dqn.load_weights(weights_filename)
history = dqn.test(env, nb_episodes=10, visualize=False)
# Obtener media y desviación estandar de la recompensa de los episodios
recompensas = history.history.get('episode_reward', [])
print(f"Media de la recompensa obtenida en 10 episodios: {np.mean(np.array(recompensas))}")
print(f"Desviación estandar de la recompensa obtenida en 10 episodios: {np.std(np.array(recompensas))}")

Testing for 10 episodes ...
Episode 1: reward: 14.000, steps: 725
Episode 2: reward: 12.000, steps: 904
Episode 3: reward: 16.000, steps: 844
Episode 4: reward: 16.000, steps: 893
Episode 5: reward: 17.000, steps: 881
Episode 6: reward: 18.000, steps: 1006
Episode 7: reward: 13.000, steps: 972
Episode 8: reward: 12.000, steps: 762
Episode 9: reward: 10.000, steps: 721
Episode 10: reward: 21.000, steps: 1634


3. Justificación de los parámetros seleccionados y de los resultados obtenidos

---